# Database Transform Testing Notebook

This notebook provides a safe environment to test database transforms without modifying the original `tgvmax.db` file.

## Features:
- Works with a copy of the original database
- Easy rollback to original state
- Test transforms before applying to production
- Compare before/after data


## Setup


In [42]:
# Import required libraries
import pandas as pd
import shutil
import os
from sqlalchemy import create_engine, text
from datetime import datetime
import sys
import time

# Add the src directory to Python path
current_dir = os.getcwd()
src_path = os.path.join(current_dir, '..', 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

try:
    from utils import remove_past_trips
    print("✅ Successfully imported utils")
except ImportError as e:
    print(f"⚠️ Could not import utils: {e}")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Setup completed")


✅ Successfully imported utils
✅ Setup completed


## Database Setup


In [43]:
# Database paths
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    ORIGINAL_DB = '../data/tgvmax.db'
    TEST_DB = '../data/tgvmax_test.db'
else:
    ORIGINAL_DB = 'data/tgvmax.db'
    TEST_DB = 'data/tgvmax_test.db'

def create_test_database():
    """Create a test copy of the original database"""
    if os.path.exists(ORIGINAL_DB):
        shutil.copy2(ORIGINAL_DB, TEST_DB)
        print(f"✅ Created test database: {TEST_DB}")
        return True
    else:
        print(f"❌ Original database not found: {ORIGINAL_DB}")
        return False

def reset_test_database():
    """Reset test database to original state"""
    if os.path.exists(ORIGINAL_DB):
        shutil.copy2(ORIGINAL_DB, TEST_DB)
        print(f"✅ Reset test database")
        return True
    else:
        print(f"❌ Cannot reset: original database not found")
        return False

# Create test database and connect
if create_test_database():
    test_engine = create_engine(f'sqlite:///{TEST_DB}')
    
    # Test connection
    with test_engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        row_count = result.fetchone()[0]
        print(f"✅ Connected to test database: {row_count:,} rows")
else:
    test_engine = None


✅ Created test database: ../data/tgvmax_test.db
✅ Connected to test database: 73,976 rows


## Database Info


In [44]:
# Show sample data
sample_df = pd.read_sql("SELECT * FROM TGVMAX LIMIT 3", con=test_engine)
display(sample_df)

# Show basic stats
with test_engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
    total_rows = result.fetchone()[0]
    
    result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
    dispo_counts = dict(result.fetchall())
    
    result = conn.execute(text("SELECT COUNT(DISTINCT DATE) FROM TGVMAX"))
    unique_dates = result.fetchone()[0]

print(f"📊 Total rows: {total_rows:,}")
print(f"📊 Availability: {dispo_counts}")
print(f"📊 Unique dates: {unique_dates}")


,date,train_no,entity,axe,origine_iata,destination_iata,origine,destination,heure_depart,heure_arrivee,DISPO,UID
0,2025-09-22,3624,TOLIPA,IC ARO,FRLST,FRPAZ,LA SOUTERRAINE,PARIS (intramuros),06:54,09:55,OUI,0
1,2025-09-23,3624,TOLIPA,IC ARO,FRSOU,FRBVE,SOUILLAC,BRIVE LA GAILLARDE,04:54,05:17,OUI,4
2,2025-09-23,3624,TOLIPA,IC ARO,FRSOU,FRCHR,SOUILLAC,CHATEAUROUX,04:54,07:30,OUI,8


📊 Total rows: 73,976
📊 Availability: {'OUI': 73976}
📊 Unique dates: 31


## Transform Testing

Add your database transforms here.


In [45]:
# Take before snapshot
def get_stats(engine, label=""):
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        total = result.fetchone()[0]
        
        result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
        dispo = dict(result.fetchall())
        
    print(f"📊 {label} - Rows: {total:,}, Available: {dispo.get('OUI', 0):,}, Unavailable: {dispo.get('NON', 0):,}")
    return total

before_count = get_stats(test_engine, "BEFORE")


📊 BEFORE - Rows: 73,976, Available: 73,976, Unavailable: 0


In [46]:
# Example transform: Test the remove_past_trips function from utils
# removed = remove_past_trips(test_engine)

# Or define your own transform here:
def example_transform(engine):
    """Example: Remove trips that have already departed"""
    start_time = time.perf_counter()
    now = datetime.now()
    today_str = now.strftime('%Y-%m-%d')
    current_time_str = now.strftime('%H:%M')

    with engine.begin() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        before_count = result.fetchone()[0]

        # Remove past trips
        delete_query = text("""
            DELETE FROM TGVMAX 
            WHERE DATE < :today 
               OR (DATE = :today AND heure_depart < :current_time)
        """)
        conn.execute(delete_query, {"today": today_str, "current_time": current_time_str})

        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        after_count = result.fetchone()[0]

        removed = before_count - after_count
        elapsed = time.perf_counter() - start_time
        print(f"🧹 Removed {removed:,} past trips ({elapsed:.3f}s)")
        return removed

# Run your transform (uncomment to test)
# removed = example_transform(test_engine)


In [47]:
# Take after snapshot and compare
after_count = get_stats(test_engine, "AFTER")
print(f"\n🔍 Change: {after_count - before_count:+,} rows")

# Basic validation
with test_engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX WHERE DATE IS NULL OR origine IS NULL"))
    null_count = result.fetchone()[0]
    
    result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX WHERE DISPO NOT IN ('OUI', 'NON')"))
    invalid_count = result.fetchone()[0]

if null_count == 0 and invalid_count == 0:
    print("✅ Data validation passed")
else:
    print(f"⚠️ Found {null_count} null values, {invalid_count} invalid DISPO values")


📊 AFTER - Rows: 73,976, Available: 73,976, Unavailable: 0

🔍 Change: +0 rows
✅ Data validation passed


## Soudure Non Autorisée Detection

A "soudure non autorisée" occurs when:
- Trip A→C is unavailable (DISPO='NON')
- But on the same train (same date, same train_no), segments A→B and B→C are available (DISPO='OUI')
- This suggests passengers could "join" the two segments to complete the A→C journey


In [48]:
import time

def find_soudure_non_autorisee_examples(engine, limit=10, verbose=True):
    """
    Find examples of 'soudure non autorisée' where:
    - A direct trip A->C is unavailable (DISPO='NON')  
    - But segments A->B and B->C are both available (DISPO='OUI') on the same train
    Times the query execution.
    """
    query = """
    WITH unavailable_direct AS (
        -- Find all unavailable direct trips A->C
        SELECT date, train_no, origine as A, destination as C, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX 
        WHERE DISPO = 'NON'
    ),
    available_segments AS (
        -- Find all available segments  
        SELECT date, train_no, origine, destination, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX 
        WHERE DISPO = 'OUI'
    )
    SELECT 
        ud.date,
        ud.train_no,
        ud.A as origine_finale,
        ud.C as destination_finale,
        ud.heure_depart as direct_depart,
        ud.heure_arrivee as direct_arrivee,
        ud.axe,
        -- First segment A->B
        seg1.destination as gare_intermediaire,
        seg1.heure_depart as segment1_depart, 
        seg1.heure_arrivee as segment1_arrivee,
        -- Second segment B->C  
        seg2.heure_depart as segment2_depart,
        seg2.heure_arrivee as segment2_arrivee
    FROM unavailable_direct ud
    -- Join with first segment A->B
    JOIN available_segments seg1 ON (
        ud.date = seg1.date 
        AND ud.train_no = seg1.train_no
        AND ud.A = seg1.origine
        AND seg1.destination != ud.C  -- B is not the final destination
    )
    -- Join with second segment B->C
    JOIN available_segments seg2 ON (
        ud.date = seg2.date
        AND ud.train_no = seg2.train_no  
        AND seg1.destination = seg2.origine  -- B matches
        AND seg2.destination = ud.C  -- Ends at final destination C
    )
    -- Ensure temporal consistency: seg1 arrival <= seg2 departure
    WHERE seg1.heure_arrivee <= seg2.heure_depart
    ORDER BY ud.date, ud.train_no, ud.heure_depart
    """
    start = time.perf_counter()
    df = pd.read_sql(query, con=engine, params={'limit': limit})
    elapsed = time.perf_counter() - start
    if verbose:
        print(f"⏱️ Query executed in {elapsed:.3f} seconds")
    return df

# Find examples of soudure non autorisée
print("🔍 Searching for examples of 'soudure non autorisée'...")
start_total = time.perf_counter()
examples = find_soudure_non_autorisee_examples(test_engine, limit=10, verbose=True)
elapsed_total = time.perf_counter() - start_total

if len(examples) > 0:
    print(f"✅ Found {len(examples)} examples of soudure non autorisée:")
    print("\n" + "="*120)
    
    for idx, row in examples.iterrows():
        print(f"📅 {row['date']} - Train {row['train_no']} ({row['axe']})")
        print(f"   ❌ DIRECT: {row['origine_finale']} → {row['destination_finale']} "
              f"({row['direct_depart']}-{row['direct_arrivee']}) - UNAVAILABLE")
        print(f"   ✅ SEGMENT 1: {row['origine_finale']} → {row['gare_intermediaire']} "
              f"({row['segment1_depart']}-{row['segment1_arrivee']}) - AVAILABLE")
        print(f"   ✅ SEGMENT 2: {row['gare_intermediaire']} → {row['destination_finale']} "
              f"({row['segment2_depart']}-{row['segment2_arrivee']}) - AVAILABLE")
        print(f"   💡 Passengers could book both segments to complete the journey!")
        print("-" * 120)
        
    # Show summary statistics
    print(f"\n📊 Summary:")
    print(f"   • Total examples found: {len(examples)}")
    print(f"   • Unique trains affected: {examples['train_no'].nunique()}")
    print(f"   • Unique dates: {examples['date'].nunique()}")
    print(f"   • Most common axes: {examples['axe'].value_counts().head(3).to_dict()}")
    print(f"⏱️ Total elapsed (including print/processing): {elapsed_total:.3f} seconds")
    
    # Display as DataFrame for detailed analysis
    display(examples[['date', 'train_no', 'origine_finale', 'gare_intermediaire', 
                     'destination_finale', 'direct_depart', 'direct_arrivee', 'axe']])
else:
    print("❌ No examples of soudure non autorisée found in current dataset")
    print("   This could mean:")
    print("   • The data is consistent (good!)")
    print("   • The detection logic needs refinement") 
    print("   • The sample size is too small")
    print(f"⏱️ Total elapsed (including print/processing): {elapsed_total:.3f} seconds")


🔍 Searching for examples of 'soudure non autorisée'...


⏱️ Query executed in 0.096 seconds
❌ No examples of soudure non autorisée found in current dataset
   This could mean:
   • The data is consistent (good!)
   • The detection logic needs refinement
   • The sample size is too small
⏱️ Total elapsed (including print/processing): 0.097 seconds


## Fix Soudure Non Autorisée

Now let's create a function to fix the detected issues by changing the availability from 'NON' to 'OUI'.


In [49]:
def fix_soudure_non_autorisee_iterative(engine):
    """
    Fix soudure non autorisée by changing DISPO from 'NON' to 'OUI' 
    for A->C trips where both A->B and B->C segments are available.
    
    Runs iteratively until no more issues are found, since fixing one 
    issue might unlock others.
    
    Args:
        engine: Database engine
    
    Returns:
        dict: Summary of all changes made across all iterations
    """
    overall_start_time = time.perf_counter()
    
    # Query to find soudure non autorisée cases
    find_query = """
    WITH unavailable_direct AS (
        -- Find all unavailable direct trips A->C
        SELECT date, train_no, origine as A, destination as C, 
               heure_depart, heure_arrivee, axe, UID
        FROM TGVMAX 
        WHERE DISPO = 'NON'
    ),
    available_segments AS (
        -- Find all available segments  
        SELECT date, train_no, origine, destination, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX 
        WHERE DISPO = 'OUI'
    )
    SELECT DISTINCT
        ud.UID as direct_uid,
        ud.date,
        ud.train_no,
        ud.A as origine_finale,
        ud.C as destination_finale,
        ud.heure_depart,
        ud.heure_arrivee,
        ud.axe
    FROM unavailable_direct ud
    -- Join with first segment A->B
    JOIN available_segments seg1 ON (
        ud.date = seg1.date 
        AND ud.train_no = seg1.train_no
        AND ud.A = seg1.origine
        AND seg1.destination != ud.C  -- B is not the final destination
    )
    -- Join with second segment B->C
    JOIN available_segments seg2 ON (
        ud.date = seg2.date
        AND ud.train_no = seg2.train_no  
        AND seg1.destination = seg2.origine  -- B matches
        AND seg2.destination = ud.C  -- Ends at final destination C
    )
    -- Ensure temporal consistency: seg1 arrival <= seg2 departure
    WHERE seg1.heure_arrivee <= seg2.heure_depart
    ORDER BY ud.date, ud.train_no, ud.heure_depart
    """
    
    total_fixed = 0
    iteration = 0
    all_iterations = []
    
    print("🔄 Starting iterative fix for soudure non autorisée...")
    
    while True:
        iteration += 1
        iteration_start = time.perf_counter()
        
        # Find issues in this iteration
        trips_to_fix = pd.read_sql(find_query, con=engine)
        
        if len(trips_to_fix) == 0:
            print(f"\n✅ Iteration {iteration}: No more soudure non autorisée issues found!")
            break
        
        print(f"\n🔍 Iteration {iteration}: Found {len(trips_to_fix)} trips to fix")
        
        # Fix the issues - batch update all at once
        uids_to_fix = trips_to_fix['direct_uid'].tolist()
        
        with engine.begin() as conn:
            if uids_to_fix:
                uid_list = ','.join(map(str, uids_to_fix))
                batch_update_query = text(f"UPDATE TGVMAX SET DISPO = 'OUI' WHERE UID IN ({uid_list})")
                result = conn.execute(batch_update_query)
                fixed_count = result.rowcount
                
                print(f"  ✅ Fixed {fixed_count} trips in batch")
                
                # Show sample of what was fixed
                sample_size = min(3, len(trips_to_fix))
                for _, trip in trips_to_fix.head(sample_size).iterrows():
                    print(f"     • {trip['origine_finale']} → {trip['destination_finale']} "
                          f"({trip['date']}, Train {trip['train_no']})")
                
                if len(trips_to_fix) > sample_size:
                    print(f"     ... and {len(trips_to_fix) - sample_size} more")
                
                total_fixed += fixed_count
                iteration_time = time.perf_counter() - iteration_start
                
                all_iterations.append({
                    'iteration': iteration,
                    'found': len(trips_to_fix),
                    'fixed': fixed_count,
                    'time': iteration_time
                })
        
        # Safety check to prevent infinite loops
        if iteration > 10:
            print(f"\n⚠️  Stopped after {iteration} iterations to prevent infinite loop")
            break
    
    overall_elapsed = time.perf_counter() - overall_start_time
    
    print(f"\n🎉 FINAL SUMMARY:")
    print(f"   • Total iterations: {iteration}")
    print(f"   • Total trips fixed: {total_fixed}")
    print(f"   • Total time: {overall_elapsed:.3f}s")
    
    if all_iterations:
        print(f"\n📊 Iteration breakdown:")
        for iter_data in all_iterations:
            print(f"   • Iteration {iter_data['iteration']}: "
                  f"{iter_data['fixed']} fixes in {iter_data['time']:.3f}s")
    
    return {
        'total_iterations': iteration,
        'total_fixed': total_fixed,
        'total_elapsed': overall_elapsed,
        'iterations': all_iterations,
        'message': f'Fixed {total_fixed} soudure non autorisée issues across {len(all_iterations)} iterations'
    }

# Run the iterative fix
print("🚀 Running iterative fix for soudure non autorisée...")
result = fix_soudure_non_autorisee_iterative(test_engine)


🚀 Running iterative fix for soudure non autorisée...
🔄 Starting iterative fix for soudure non autorisée...



✅ Iteration 1: No more soudure non autorisée issues found!

🎉 FINAL SUMMARY:
   • Total iterations: 1
   • Total trips fixed: 0
   • Total time: 0.126s


In [50]:
# Debug: Let's investigate the discrepancy between examples found and fixes applied
print("🔍 DEBUGGING: Analyzing soudure detection vs fix discrepancy...")

# Get the examples without limit to see all cases
all_examples = find_soudure_non_autorisee_examples(test_engine, limit=None, verbose=False)
print(f"📊 Total examples found by detection function: {len(all_examples)}")

# Check for duplicates in the examples
if len(all_examples) > 0:
    unique_uids = all_examples['direct_uid'].nunique()
    print(f"📊 Unique UIDs in examples: {unique_uids}")
    print(f"📊 Duplicate ratio: {len(all_examples) - unique_uids} duplicates out of {len(all_examples)} total")
    
    # Show some duplicates if they exist
    if len(all_examples) > unique_uids:
        print("\n🔍 Sample of duplicate UIDs:")
        duplicates = all_examples[all_examples.duplicated(subset=['direct_uid'], keep=False)]
        for uid in duplicates['direct_uid'].unique()[:3]:
            uid_examples = duplicates[duplicates['direct_uid'] == uid]
            print(f"   UID {uid} appears {len(uid_examples)} times:")
            for _, row in uid_examples.head(2).iterrows():
                print(f"     • {row['origine_finale']} → {row['destination_finale']} via {row['gare_intermediaire']}")
    
    # Check if some UIDs don't exist in the database anymore
    print(f"\n🔍 Checking if all UIDs exist in database...")
    uids_to_check = all_examples['direct_uid'].unique()[:10]  # Check first 10
    
    with test_engine.connect() as conn:
        uid_list = ','.join(map(str, uids_to_check))
        check_query = text(f"SELECT UID, DISPO FROM TGVMAX WHERE UID IN ({uid_list})")
        existing_uids = pd.read_sql(check_query, con=conn)
    
    print(f"   • Checked {len(uids_to_check)} UIDs from examples")
    print(f"   • Found {len(existing_uids)} UIDs in database")
    
    if len(existing_uids) < len(uids_to_check):
        missing_uids = set(uids_to_check) - set(existing_uids['UID'])
        print(f"   • Missing UIDs: {missing_uids}")
    
    # Check current DISPO status of sample UIDs
    available_count = len(existing_uids[existing_uids['DISPO'] == 'OUI'])
    unavailable_count = len(existing_uids[existing_uids['DISPO'] == 'NON'])
    print(f"   • Current status: {available_count} available, {unavailable_count} unavailable")

else:
    print("❌ No examples found to analyze")


🔍 DEBUGGING: Analyzing soudure detection vs fix discrepancy...


📊 Total examples found by detection function: 0
❌ No examples found to analyze


In [51]:
# Debug: Let's also check what the fix function is actually trying to update
print("\n🔍 DEBUGGING: Analyzing what the fix function finds...")

# Let's manually run the fix query to see what it finds
fix_query = """
WITH unavailable_direct AS (
    -- Find all unavailable direct trips A->C
    SELECT date, train_no, origine as A, destination as C, 
           heure_depart, heure_arrivee, axe, UID
    FROM TGVMAX 
    WHERE DISPO = 'NON'
),
available_segments AS (
    -- Find all available segments  
    SELECT date, train_no, origine, destination, 
           heure_depart, heure_arrivee, axe
    FROM TGVMAX 
    WHERE DISPO = 'OUI'
)
SELECT DISTINCT
    ud.UID as direct_uid,
    ud.date,
    ud.train_no,
    ud.A as origine_finale,
    ud.C as destination_finale,
    ud.heure_depart,
    ud.heure_arrivee,
    ud.axe
FROM unavailable_direct ud
-- Join with first segment A->B
JOIN available_segments seg1 ON (
    ud.date = seg1.date 
    AND ud.train_no = seg1.train_no
    AND ud.A = seg1.origine
    AND seg1.destination != ud.C  -- B is not the final destination
)
-- Join with second segment B->C
JOIN available_segments seg2 ON (
    ud.date = seg2.date
    AND ud.train_no = seg2.train_no  
    AND seg1.destination = seg2.origine  -- B matches
    AND seg2.destination = ud.C  -- Ends at final destination C
)
-- Ensure temporal consistency: seg1 arrival <= seg2 departure
WHERE seg1.heure_arrivee <= seg2.heure_depart
ORDER BY ud.date, ud.train_no, ud.heure_depart
"""

fix_examples = pd.read_sql(fix_query, con=test_engine)
print(f"📊 Fix function finds: {len(fix_examples)} trips to fix")
print(f"📊 Fix function unique UIDs: {fix_examples['direct_uid'].nunique()}")

# Compare the two queries
print(f"\n🔍 COMPARISON:")
print(f"   • Detection function: {len(all_examples)} examples ({all_examples['direct_uid'].nunique() if len(all_examples) > 0 else 0} unique UIDs)")
print(f"   • Fix function: {len(fix_examples)} examples ({fix_examples['direct_uid'].nunique()} unique UIDs)")

# Check if the UIDs are the same
if len(all_examples) > 0 and len(fix_examples) > 0:
    detection_uids = set(all_examples['direct_uid'].unique())
    fix_uids = set(fix_examples['direct_uid'].unique())
    
    common_uids = detection_uids.intersection(fix_uids)
    only_in_detection = detection_uids - fix_uids
    only_in_fix = fix_uids - detection_uids
    
    print(f"   • Common UIDs: {len(common_uids)}")
    print(f"   • Only in detection: {len(only_in_detection)}")
    print(f"   • Only in fix: {len(only_in_fix)}")
    
    if only_in_detection:
        print(f"   • Sample UIDs only in detection: {list(only_in_detection)[:5]}")
    if only_in_fix:
        print(f"   • Sample UIDs only in fix: {list(only_in_fix)[:5]}")

# The discrepancy might be due to:
# 1. Duplicates in the detection function
# 2. Different SQL logic between detection and fix
# 3. UIDs that don't exist or were already fixed



🔍 DEBUGGING: Analyzing what the fix function finds...


📊 Fix function finds: 0 trips to fix
📊 Fix function unique UIDs: 0

🔍 COMPARISON:
   • Detection function: 0 examples (0 unique UIDs)
   • Fix function: 0 examples (0 unique UIDs)


In [52]:
# Final verification - check if any soudure non autorisée issues remain
print("🔍 Final verification: Checking for any remaining soudure non autorisée issues...")
remaining_issues = find_soudure_non_autorisee_examples(test_engine, limit=5, verbose=False)

if len(remaining_issues) == 0:
    print("🎉 SUCCESS: No remaining soudure non autorisée issues found!")
    print("   All database inconsistencies have been resolved.")
else:
    print(f"⚠️  WARNING: {len(remaining_issues)} issues still remain")
    print("   These may require manual investigation:")
    for _, issue in remaining_issues.head(3).iterrows():
        print(f"     • {issue['origine_finale']} → {issue['destination_finale']} "
              f"(Train {issue['train_no']}, {issue['date']})")


🔍 Final verification: Checking for any remaining soudure non autorisée issues...
🎉 SUCCESS: No remaining soudure non autorisée issues found!
   All database inconsistencies have been resolved.


## Coupure Non Autorisée Detection & Fix

A "coupure non autorisée" occurs when:
- Trip A→B is unavailable (DISPO='NON')
- But on the same train (same date, same train_no), trip A→C is available (DISPO='OUI')
- And B is an intermediate station between A and C (B's arrival time is between A's departure and C's arrival)
- This suggests passengers could "cut short" their journey at station B


In [53]:
def find_coupure_non_autorisee_examples(engine, limit=10, verbose=True):
    """
    Find examples of 'coupure non autorisée' where:
    - A trip A->B is unavailable (DISPO='NON')
    - But A->C is available (DISPO='OUI') on the same train
    - And B is an intermediate station (arrives between A's departure and C's arrival)
    """
    
    query = """
    WITH unavailable_short AS (
        -- Find all unavailable short trips A->B
        SELECT date, train_no, origine as A, destination as B, 
               heure_depart, heure_arrivee, axe, UID
        FROM TGVMAX 
        WHERE DISPO = 'NON'
    ),
    available_long AS (
        -- Find all available longer trips A->C
        SELECT date, train_no, origine as A, destination as C, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX 
        WHERE DISPO = 'OUI'
    ),
    intermediate_stations AS (
        -- Find all stations B that are intermediate stops on A->C routes
        SELECT DISTINCT date, train_no, origine, destination, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX
    )
    SELECT 
        us.date,
        us.train_no,
        us.A as origine_finale,
        us.B as destination_intermediate,
        al.C as destination_finale,
        us.heure_depart as short_depart,
        us.heure_arrivee as short_arrivee,
        al.heure_depart as long_depart,
        al.heure_arrivee as long_arrivee,
        inter.heure_arrivee as intermediate_arrivee,
        us.axe,
        us.UID as short_uid
    FROM unavailable_short us
    -- Join with available longer trip A->C (same origin)
    JOIN available_long al ON (
        us.date = al.date 
        AND us.train_no = al.train_no
        AND us.A = al.A  -- Same origin
        AND us.B != al.C  -- Different destinations
    )
    -- Join to find B as intermediate station on the A->C route
    JOIN intermediate_stations inter ON (
        us.date = inter.date
        AND us.train_no = inter.train_no
        AND us.B = inter.destination  -- B is a destination on this train
        AND inter.origine = us.A      -- Starting from A
    )
    -- Ensure B is truly intermediate: A depart < B arrive < C arrive
    WHERE us.heure_depart = al.heure_depart  -- Same departure time from A
      AND inter.heure_arrivee > us.heure_depart   -- B arrival after A departure
      AND inter.heure_arrivee < al.heure_arrivee  -- B arrival before C arrival
      AND us.heure_arrivee = inter.heure_arrivee  -- Consistent B arrival time
    ORDER BY us.date, us.train_no, us.heure_depart
    """

    
    start_time = time.perf_counter()
    examples = pd.read_sql(query, con=engine)
    elapsed = time.perf_counter() - start_time
    
    if verbose:
        if len(examples) > 0:
            print(f"✅ Found {len(examples)} examples of coupure non autorisée:")
            print("\n" + "="*120)
            
            for idx, row in examples.iterrows():
                print(f"📅 {row['date']} - Train {row['train_no']} ({row['axe']})")
                print(f"   ❌ SHORT: {row['origine_finale']} → {row['destination_intermediate']} "
                      f"({row['short_depart']}-{row['short_arrivee']}) - UID: {row['short_uid']} - UNAVAILABLE")
                print(f"   ✅ LONG: {row['origine_finale']} → {row['destination_finale']} "
                      f"({row['long_depart']}-{row['long_arrivee']}) - AVAILABLE")
                print(f"   💡 Station {row['destination_intermediate']} is intermediate - passengers could get off early!")
                print("-" * 120)
                
            # Show summary statistics
            print(f"\n📊 Summary:")
            print(f"   • Total examples found: {len(examples)}")
            print(f"   • Unique trains affected: {examples['train_no'].nunique()}")
            print(f"   • Unique dates: {examples['date'].nunique()}")
            print(f"   • Query time: {elapsed:.3f}s")
            
        else:
            print("❌ No examples of coupure non autorisée found in current dataset")
            print(f"   Query completed in {elapsed:.3f}s")
    
    return examples

# Test the detection function
print("🔍 Searching for examples of 'coupure non autorisée'...")
coupure_examples = find_coupure_non_autorisee_examples(test_engine, limit=10)


🔍 Searching for examples of 'coupure non autorisée'...


❌ No examples of coupure non autorisée found in current dataset
   Query completed in 0.285s


In [54]:
def fix_coupure_non_autorisee(engine):
    """
    Fix coupure non autorisée by changing DISPO from 'NON' to 'OUI' 
    for A->B trips where A->C is available and B is intermediate.
    
    Unlike soudure, coupure fixes don't create cascading effects,
    so a single pass is sufficient.
    
    Args:
        engine: Database engine
    
    Returns:
        dict: Summary of changes made
    """
    start_time = time.perf_counter()
    
    # Query to find coupure non autorisée cases
    find_query = """
    WITH unavailable_short AS (
        -- Find all unavailable short trips A->B
        SELECT date, train_no, origine as A, destination as B, 
               heure_depart, heure_arrivee, axe, UID
        FROM TGVMAX 
        WHERE DISPO = 'NON'
    ),
    available_long AS (
        -- Find all available longer trips A->C
        SELECT date, train_no, origine as A, destination as C, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX 
        WHERE DISPO = 'OUI'
    ),
    intermediate_stations AS (
        -- Find all stations B that are intermediate stops on A->C routes
        SELECT DISTINCT date, train_no, origine, destination, 
               heure_depart, heure_arrivee, axe
        FROM TGVMAX
    )
    SELECT DISTINCT
        us.UID as short_uid,
        us.date,
        us.train_no,
        us.A as origine_finale,
        us.B as destination_intermediate,
        al.C as destination_finale,
        us.heure_depart,
        us.heure_arrivee,
        us.axe
    FROM unavailable_short us
    -- Join with available longer trip A->C (same origin)
    JOIN available_long al ON (
        us.date = al.date 
        AND us.train_no = al.train_no
        AND us.A = al.A  -- Same origin
        AND us.B != al.C  -- Different destinations
    )
    -- Join to find B as intermediate station on the A->C route
    JOIN intermediate_stations inter ON (
        us.date = inter.date
        AND us.train_no = inter.train_no
        AND us.B = inter.destination  -- B is a destination on this train
        AND inter.origine = us.A      -- Starting from A
    )
    -- Ensure B is truly intermediate: A depart < B arrive < C arrive
    WHERE us.heure_depart = al.heure_depart  -- Same departure time from A
      AND inter.heure_arrivee > us.heure_depart   -- B arrival after A departure
      AND inter.heure_arrivee < al.heure_arrivee  -- B arrival before C arrival
      AND us.heure_arrivee = inter.heure_arrivee  -- Consistent B arrival time
    ORDER BY us.date, us.train_no, us.heure_depart
    """
    
    print("🔍 Finding coupure non autorisée issues...")
    trips_to_fix = pd.read_sql(find_query, con=engine)
    
    if len(trips_to_fix) == 0:
        elapsed = time.perf_counter() - start_time
        print("✅ No coupure non autorisée issues found!")
        return {
            'found': 0,
            'fixed': 0,
            'elapsed': elapsed,
            'message': 'No coupure non autorisée issues found'
        }
    
    print(f"🔍 Found {len(trips_to_fix)} trips with coupure non autorisée")
    
    # Fix all issues in a single batch update
    uids_to_fix = trips_to_fix['short_uid'].tolist()
    
    with engine.begin() as conn:
        uid_list = ','.join(map(str, uids_to_fix))
        batch_update_query = text(f"UPDATE TGVMAX SET DISPO = 'OUI' WHERE UID IN ({uid_list})")
        result = conn.execute(batch_update_query)
        fixed_count = result.rowcount
        
        print(f"🔧 Fixed {fixed_count} trips in single batch operation")
        
        # Show sample of what was fixed
        sample_size = min(5, len(trips_to_fix))
        print(f"\n📋 Sample of fixes applied:")
        for _, trip in trips_to_fix.head(sample_size).iterrows():
            print(f"   • {trip['origine_finale']} → {trip['destination_intermediate']} "
                  f"(Train {trip['train_no']}, {trip['date']})")
            print(f"     ↳ Now available because {trip['origine_finale']} → {trip['destination_finale']} was already available")
        
        if len(trips_to_fix) > sample_size:
            print(f"   ... and {len(trips_to_fix) - sample_size} more")
    
    elapsed = time.perf_counter() - start_time
    
    print(f"\n✅ SUMMARY:")
    print(f"   • Issues found: {len(trips_to_fix)}")
    print(f"   • Trips fixed: {fixed_count}")
    print(f"   • Time taken: {elapsed:.3f}s")
    print(f"   • Single-pass operation (no iterations needed)")
    
    return {
        'found': len(trips_to_fix),
        'fixed': fixed_count,
        'elapsed': elapsed,
        'message': f'Fixed {fixed_count} coupure non autorisée issues in single operation'
    }

# Run the single-pass fix for coupure non autorisée
print("🚀 Running single-pass fix for coupure non autorisée...")
coupure_result = fix_coupure_non_autorisee(test_engine)


🚀 Running single-pass fix for coupure non autorisée...
🔍 Finding coupure non autorisée issues...
✅ No coupure non autorisée issues found!


In [55]:
# Final verification - check if any coupure non autorisée issues remain
print("🔍 Final verification: Checking for any remaining coupure non autorisée issues...")
remaining_coupure_issues = find_coupure_non_autorisee_examples(test_engine, limit=5, verbose=False)

if len(remaining_coupure_issues) == 0:
    print("🎉 SUCCESS: No remaining coupure non autorisée issues found!")
    print("   All short-trip inconsistencies have been resolved.")
else:
    print(f"⚠️  WARNING: {len(remaining_coupure_issues)} issues still remain")
    print("   These may require manual investigation:")
    for _, issue in remaining_coupure_issues.head(3).iterrows():
        print(f"     • {issue['origine_finale']} → {issue['destination_intermediate']} "
              f"(Train {issue['train_no']}, {issue['date']})")
        print(f"       (Long trip {issue['origine_finale']} → {issue['destination_finale']} is available)")


🔍 Final verification: Checking for any remaining coupure non autorisée issues...
🎉 SUCCESS: No remaining coupure non autorisée issues found!
   All short-trip inconsistencies have been resolved.


## Database Cleanup: Remove Unavailable Trips

After fixing all the inconsistencies, we can safely remove all trips that are still unavailable (DISPO='NON') to optimize the database size and performance.


In [56]:
def cleanup_unavailable_trips(engine, confirm_delete=True):
    """
    Remove all trips with DISPO='NON' from the database.
    
    This should be run AFTER fixing soudure and coupure non autorisée
    to ensure we don't accidentally delete trips that should be available.
    
    Args:
        engine: Database engine
        confirm_delete: If True, shows what will be deleted before proceeding
    
    Returns:
        dict: Summary of cleanup operation
    """
    start_time = time.perf_counter()
    
    print("🧹 DATABASE CLEANUP: Removing unavailable trips...")
    
    # First, get statistics about what we're about to delete
    with engine.connect() as conn:
        # Total trips before cleanup
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        total_before = result.fetchone()[0]
        
        # Count unavailable trips
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX WHERE DISPO = 'NON'"))
        unavailable_count = result.fetchone()[0]
        
        # Count available trips (these will remain)
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX WHERE DISPO = 'OUI'"))
        available_count = result.fetchone()[0]
        
        # Get some sample unavailable trips for display
        sample_unavailable = pd.read_sql("""
            SELECT date, train_no, origine, destination, heure_depart, heure_arrivee, axe
            FROM TGVMAX 
            WHERE DISPO = 'NON' 
            ORDER BY date, train_no 
            LIMIT 5
        """, con=conn)
    
    print(f"📊 CLEANUP ANALYSIS:")
    print(f"   • Total trips in database: {total_before:,}")
    print(f"   • Available trips (will keep): {available_count:,} ({available_count/total_before*100:.1f}%)")
    print(f"   • Unavailable trips (will delete): {unavailable_count:,} ({unavailable_count/total_before*100:.1f}%)")
    print(f"   • Database size reduction: ~{unavailable_count/total_before*100:.1f}%")
    
    if unavailable_count > 0:
        print(f"\n📋 Sample trips to be deleted:")
        for _, trip in sample_unavailable.iterrows():
            print(f"   • {trip['date']} Train {trip['train_no']}: "
                  f"{trip['origine']} → {trip['destination']} "
                  f"({trip['heure_depart']}-{trip['heure_arrivee']}) [{trip['axe']}]")
        
        if len(sample_unavailable) < unavailable_count:
            print(f"   ... and {unavailable_count - len(sample_unavailable):,} more trips")
    
    if confirm_delete and unavailable_count > 0:
        print(f"\n⚠️  WARNING: This will permanently delete {unavailable_count:,} trips!")
        print("   Make sure you have:")
        print("   • ✅ Fixed all soudure non autorisée issues")
        print("   • ✅ Fixed all coupure non autorisée issues") 
        print("   • ✅ Backed up your database if needed")
        print(f"\n   To proceed with deletion, run:")
        print(f"   cleanup_result = cleanup_unavailable_trips(test_engine, confirm_delete=False)")
        
        return {
            'total_before': total_before,
            'available': available_count,
            'unavailable': unavailable_count,
            'deleted': 0,
            'total_after': total_before,
            'message': f'Cleanup ready: {unavailable_count:,} trips can be deleted',
            'confirmed': False
        }
    
    if unavailable_count == 0:
        elapsed = time.perf_counter() - start_time
        print("✅ No unavailable trips found - database is already clean!")
        return {
            'total_before': total_before,
            'available': available_count,
            'unavailable': 0,
            'deleted': 0,
            'total_after': total_before,
            'elapsed': elapsed,
            'message': 'Database already clean - no trips to delete',
            'confirmed': True
        }
    
    # Perform the actual deletion
    print(f"\n🗑️  DELETING {unavailable_count:,} unavailable trips...")
    
    with engine.begin() as conn:
        delete_query = text("DELETE FROM TGVMAX WHERE DISPO = 'NON'")
        result = conn.execute(delete_query)
        deleted_count = result.rowcount
        
        # Get final count
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        total_after = result.fetchone()[0]
    
    elapsed = time.perf_counter() - start_time
    
    print(f"✅ CLEANUP COMPLETED!")
    print(f"   • Deleted: {deleted_count:,} trips")
    print(f"   • Remaining: {total_after:,} trips")
    print(f"   • Database reduction: {((total_before - total_after) / total_before * 100):.1f}%")
    print(f"   • Time taken: {elapsed:.3f}s")
    print(f"   • All remaining trips are available (DISPO='OUI')")
    
    return {
        'total_before': total_before,
        'available': available_count,
        'unavailable': unavailable_count,
        'deleted': deleted_count,
        'total_after': total_after,
        'reduction_percent': (total_before - total_after) / total_before * 100,
        'elapsed': elapsed,
        'message': f'Successfully deleted {deleted_count:,} unavailable trips',
        'confirmed': True
    }

# First run with confirmation to see what would be deleted
print("🔍 Analyzing database for cleanup opportunities...")
cleanup_analysis = cleanup_unavailable_trips(test_engine, confirm_delete=True)


🔍 Analyzing database for cleanup opportunities...
🧹 DATABASE CLEANUP: Removing unavailable trips...
📊 CLEANUP ANALYSIS:
   • Total trips in database: 73,976
   • Available trips (will keep): 73,976 (100.0%)
   • Unavailable trips (will delete): 0 (0.0%)
   • Database size reduction: ~0.0%
✅ No unavailable trips found - database is already clean!


In [57]:
# To actually perform the cleanup, uncomment and run:
# print("🗑️  Performing database cleanup...")
# cleanup_result = cleanup_unavailable_trips(test_engine, confirm_delete=False)

# Verify the cleanup results
print("\n🔍 Final database verification after all operations...")

with test_engine.connect() as conn:
    # Get final statistics
    result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
    final_total = result.fetchone()[0]
    
    result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
    final_stats = dict(result.fetchall())
    
    # Get date range
    result = conn.execute(text("SELECT MIN(date) as min_date, MAX(date) as max_date FROM TGVMAX"))
    date_range = result.fetchone()
    
    # Get unique trains and stations
    result = conn.execute(text("SELECT COUNT(DISTINCT train_no) FROM TGVMAX"))
    unique_trains = result.fetchone()[0]
    
    result = conn.execute(text("""
        SELECT COUNT(DISTINCT station) FROM (
            SELECT origine as station FROM TGVMAX 
            UNION 
            SELECT destination as station FROM TGVMAX
        )
    """))
    unique_stations = result.fetchone()[0]

print(f"📊 FINAL DATABASE STATE:")
print(f"   • Total trips: {final_total:,}")
print(f"   • Available (OUI): {final_stats.get('OUI', 0):,}")
print(f"   • Unavailable (NON): {final_stats.get('NON', 0):,}")
print(f"   • Availability rate: {(final_stats.get('OUI', 0) / final_total * 100):.1f}%")
print(f"   • Date range: {date_range[0]} to {date_range[1]}")
print(f"   • Unique trains: {unique_trains:,}")
print(f"   • Unique stations: {unique_stations:,}")

if final_stats.get('NON', 0) == 0:
    print(f"\n🎉 PERFECT! Database contains only available trips!")
    print(f"   • 100% of trips are bookable by TGV Max users")
    print(f"   • Database is optimized for maximum performance")
    print(f"   • All inconsistencies have been resolved")
else:
    print(f"\n⚠️  Note: {final_stats.get('NON', 0):,} unavailable trips remain")
    print(f"   Run the cleanup function to remove them if desired")



🔍 Final database verification after all operations...


📊 FINAL DATABASE STATE:
   • Total trips: 73,976
   • Available (OUI): 73,976
   • Unavailable (NON): 0
   • Availability rate: 100.0%
   • Date range: 2025-09-21 to 2025-10-21
   • Unique trains: 834
   • Unique stations: 220

🎉 PERFECT! Database contains only available trips!
   • 100% of trips are bookable by TGV Max users
   • Database is optimized for maximum performance
   • All inconsistencies have been resolved


## Master Database Optimization Function

This function runs the complete optimization pipeline: coupure → soudure → cleanup, with comprehensive before/after reporting.


In [58]:
def optimize_database_complete(engine, perform_cleanup=True):
    """
    Complete database optimization pipeline:
    1. Fix coupure non autorisée (single pass)
    2. Fix soudure non autorisée (iterative)
    3. Clean up unavailable trips (optional)
    
    Args:
        engine: Database engine
        perform_cleanup: If True, removes all remaining unavailable trips
    
    Returns:
        dict: Comprehensive summary of all optimizations
    """
    overall_start = time.perf_counter()
    
    print("🚀 STARTING COMPLETE DATABASE OPTIMIZATION")
    print("=" * 60)
    
    # Get initial database state
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        initial_total = result.fetchone()[0]
        
        result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
        initial_stats = dict(result.fetchall())
        
        initial_available = initial_stats.get('OUI', 0)
        initial_unavailable = initial_stats.get('NON', 0)
        initial_availability_rate = (initial_available / initial_total * 100) if initial_total > 0 else 0
    
    print(f"📊 INITIAL STATE:")
    print(f"   • Total trips: {initial_total:,}")
    print(f"   • Available: {initial_available:,} ({initial_availability_rate:.1f}%)")
    print(f"   • Unavailable: {initial_unavailable:,} ({100-initial_availability_rate:.1f}%)")
    
    # STEP 1: Fix coupure non autorisée
    print(f"\n" + "="*60)
    print("🔧 STEP 1: Fixing coupure non autorisée...")
    coupure_result = fix_coupure_non_autorisee(engine)
    coupure_fixes = coupure_result.get('fixed', 0)
    
    # STEP 2: Fix soudure non autorisée  
    print(f"\n" + "="*60)
    print("🔧 STEP 2: Fixing soudure non autorisée...")
    soudure_result = fix_soudure_non_autorisee_iterative(engine)
    soudure_fixes = soudure_result.get('total_fixed', 0)
    soudure_iterations = soudure_result.get('total_iterations', 0)
    
    # Get state after fixes
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        after_fixes_total = result.fetchone()[0]
        
        result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
        after_fixes_stats = dict(result.fetchall())
        
        after_fixes_available = after_fixes_stats.get('OUI', 0)
        after_fixes_unavailable = after_fixes_stats.get('NON', 0)
        after_fixes_rate = (after_fixes_available / after_fixes_total * 100) if after_fixes_total > 0 else 0
    
    # STEP 3: Cleanup (optional)
    cleanup_result = None
    if perform_cleanup and after_fixes_unavailable > 0:
        print(f"\n" + "="*60)
        print("🧹 STEP 3: Cleaning up remaining unavailable trips...")
        cleanup_result = cleanup_unavailable_trips(engine, confirm_delete=False)
    elif perform_cleanup:
        print(f"\n" + "="*60)
        print("🧹 STEP 3: No cleanup needed - all trips are available!")
        cleanup_result = {'deleted': 0, 'message': 'No unavailable trips to delete'}
    else:
        print(f"\n" + "="*60)
        print("⏭️  STEP 3: Skipping cleanup (perform_cleanup=False)")
    
    # Get final state
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
        final_total = result.fetchone()[0]
        
        result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
        final_stats = dict(result.fetchall())
        
        final_available = final_stats.get('OUI', 0)
        final_unavailable = final_stats.get('NON', 0)
        final_availability_rate = (final_available / final_total * 100) if final_total > 0 else 0
    
    overall_elapsed = time.perf_counter() - overall_start
    
    # Calculate improvements
    total_new_available = final_available - initial_available
    availability_improvement = final_availability_rate - initial_availability_rate
    trips_deleted = cleanup_result.get('deleted', 0) if cleanup_result else 0
    
    # FINAL REPORT
    print(f"\n" + "🎉" + "="*58 + "🎉")
    print("🏆 OPTIMIZATION COMPLETE - FINAL REPORT")
    print("🎉" + "="*58 + "🎉")
    
    print(f"\n📈 AVAILABILITY IMPROVEMENTS:")
    print(f"   • New available trips: +{total_new_available:,}")
    if initial_available > 0:
        percent_increase = (total_new_available / initial_available * 100)
        print(f"   • Relative increase: +{percent_increase:.1f}% more available trips")
    print(f"   • Availability rate: {initial_availability_rate:.1f}% → {final_availability_rate:.1f}% (+{availability_improvement:.1f}%)")
    
    print(f"\n🔧 FIXES APPLIED:")
    print(f"   • Coupure non autorisée: {coupure_fixes:,} trips fixed")
    print(f"   • Soudure non autorisée: {soudure_fixes:,} trips fixed ({soudure_iterations} iterations)")
    print(f"   • Total fixes: {coupure_fixes + soudure_fixes:,} trips made available")
    
    print(f"\n🧹 DATABASE OPTIMIZATION:")
    print(f"   • Trips deleted: {trips_deleted:,}")
    print(f"   • Size reduction: {initial_total:,} → {final_total:,} trips")
    if initial_total > 0:
        size_reduction = ((initial_total - final_total) / initial_total * 100)
        print(f"   • Storage savings: -{size_reduction:.1f}%")
    
    print(f"\n📊 FINAL STATE:")
    print(f"   • Total trips: {final_total:,}")
    print(f"   • Available: {final_available:,} ({final_availability_rate:.1f}%)")
    print(f"   • Unavailable: {final_unavailable:,}")
    print(f"   • Total time: {overall_elapsed:.3f}s")
    
    if final_unavailable == 0:
        print(f"\n🌟 PERFECT OPTIMIZATION ACHIEVED!")
        print(f"   • 100% of remaining trips are available")
        print(f"   • Database is fully optimized for TGV Max users")
    
    return {
        # Initial state
        'initial_total': initial_total,
        'initial_available': initial_available,
        'initial_unavailable': initial_unavailable,
        'initial_availability_rate': initial_availability_rate,
        
        # Final state  
        'final_total': final_total,
        'final_available': final_available,
        'final_unavailable': final_unavailable,
        'final_availability_rate': final_availability_rate,
        
        # Improvements
        'new_available_trips': total_new_available,
        'availability_improvement': availability_improvement,
        'percent_increase': (total_new_available / initial_available * 100) if initial_available > 0 else 0,
        
        # Fixes applied
        'coupure_fixes': coupure_fixes,
        'soudure_fixes': soudure_fixes,
        'soudure_iterations': soudure_iterations,
        'total_fixes': coupure_fixes + soudure_fixes,
        
        # Cleanup
        'trips_deleted': trips_deleted,
        'size_reduction_percent': ((initial_total - final_total) / initial_total * 100) if initial_total > 0 else 0,
        
        # Performance
        'total_time': overall_elapsed,
        'steps_completed': 3 if perform_cleanup else 2,
        
        # Individual results
        'coupure_result': coupure_result,
        'soudure_result': soudure_result,
        'cleanup_result': cleanup_result,
        
        'message': f'Optimization complete: +{total_new_available:,} available trips ({availability_improvement:+.1f}% improvement)'
    }

# Run the complete optimization pipeline
print("🎯 RUNNING COMPLETE DATABASE OPTIMIZATION PIPELINE...")
optimization_result = optimize_database_complete(test_engine, perform_cleanup=True)


🎯 RUNNING COMPLETE DATABASE OPTIMIZATION PIPELINE...
🚀 STARTING COMPLETE DATABASE OPTIMIZATION
📊 INITIAL STATE:
   • Total trips: 73,976
   • Available: 73,976 (100.0%)
   • Unavailable: 0 (0.0%)

🔧 STEP 1: Fixing coupure non autorisée...
🔍 Finding coupure non autorisée issues...
✅ No coupure non autorisée issues found!

🔧 STEP 2: Fixing soudure non autorisée...
🔄 Starting iterative fix for soudure non autorisée...

✅ Iteration 1: No more soudure non autorisée issues found!

🎉 FINAL SUMMARY:
   • Total iterations: 1
   • Total trips fixed: 0
   • Total time: 0.101s

🧹 STEP 3: No cleanup needed - all trips are available!

🎉==========================================================🎉
🏆 OPTIMIZATION COMPLETE - FINAL REPORT
🎉==========================================================🎉

📈 AVAILABILITY IMPROVEMENTS:
   • New available trips: +0
   • Relative increase: +0.0% more available trips
   • Availability rate: 100.0% → 100.0% (+0.0%)

🔧 FIXES APPLIED:
   • Coupure non autorisée: 0 trip

## Summary: Database Consistency Improvements

Both soudure and coupure non autorisée issues have been detected and fixed!


In [59]:
# Combined summary of all fixes applied
print("📋 COMPLETE DATABASE CONSISTENCY REPORT")
print("=" * 60)

# Get final stats
with test_engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM TGVMAX"))
    total_trips = result.fetchone()[0]
    
    result = conn.execute(text("SELECT DISPO, COUNT(*) FROM TGVMAX GROUP BY DISPO"))
    availability_stats = dict(result.fetchall())

print(f"📊 Final Database State:")
print(f"   • Total trips: {total_trips:,}")
print(f"   • Available (OUI): {availability_stats.get('OUI', 0):,}")
print(f"   • Unavailable (NON): {availability_stats.get('NON', 0):,}")
print(f"   • Availability rate: {(availability_stats.get('OUI', 0) / total_trips * 100):.1f}%")

print(f"\n🔧 Fixes Applied:")
if 'result' in locals():
    soudure_fixed = result.get('total_fixed', 0)
    soudure_iterations = result.get('total_iterations', 0)
    print(f"   • Soudure non autorisée: {soudure_fixed} trips fixed in {soudure_iterations} iterations")

if 'coupure_result' in locals():
    coupure_fixed = coupure_result.get('fixed', 0)
    print(f"   • Coupure non autorisée: {coupure_fixed} trips fixed in single pass")

total_fixes = (result.get('total_fixed', 0) if 'result' in locals() else 0) + \
              (coupure_result.get('fixed', 0) if 'coupure_result' in locals() else 0)

print(f"\n🎉 TOTAL IMPROVEMENTS: {total_fixes} trips made available")
print("   Database consistency significantly improved!")
print("   TGV Max users now have access to more trip options.")


📋 COMPLETE DATABASE CONSISTENCY REPORT
📊 Final Database State:
   • Total trips: 73,976
   • Available (OUI): 73,976
   • Unavailable (NON): 0
   • Availability rate: 100.0%

🔧 Fixes Applied:


AttributeError: 'LegacyCursorResult' object has no attribute 'get'

## Utilities
th

In [ ]:
def quick_query(query, limit=5):
    """Execute a quick query"""
    if "LIMIT" not in query.upper():
        query += f" LIMIT {limit}"
    df = pd.read_sql(query, con=test_engine)
    display(df)
    return df

# Example: quick_query("SELECT * FROM TGVMAX WHERE DISPO = 'OUI'")


In [ ]:
# Reset database if needed
reset_test_database()
test_engine = create_engine(f'sqlite:///{TEST_DB}')
print("✅ Database reset")


✅ Reset test database
✅ Database reset
